In [1]:
from langchain.llms import Ollama
import bs4
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader

In [2]:
llm = Ollama(model="llama2")
embeddings_provider = OllamaEmbeddings()

In [3]:
loader = WebBaseLoader(
    ["https://lilianweng.github.io/posts/2023-06-23-agent/"
     ]
)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

In [5]:
vector_store = DocArrayInMemorySearch.from_documents(document_chunks, embeddings_provider)
retriever = vector_store.as_retriever()
output_parser = StrOutputParser()

In [6]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [7]:
contextualize_q_chain = contextualize_q_prompt | llm | output_parser

In [8]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [9]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
        RunnablePassthrough.assign(
            context=contextualized_question | retriever | format_docs
        )
        | qa_prompt
        | llm
)

In [11]:
chat_history = []

question = "What is a large language model?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg)
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg)])

A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. LLMs have become increasingly popular in recent years due to their ability to process and generate text at scale, and have a wide range of applications such as language translation, text summarization, and content generation.

There are several types of LLMs, including:

1. Neural Network-based Models: These models use neural networks to learn the patterns and structures of language from large datasets of text. Examples include Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, and Transformers.
2. Statistical Models: These models use statistical techniques such as probabilistic modeling and maximum likelihood estimation to learn the patterns and structures of language from large datasets of text. Examples include N-gram models and Markov Chain models.
3. Hybrid Models: These m

In [12]:
second_question = "Can you explain the reasoning behind calling it large?"
second_answer = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
print(second_answer)

AI: Certainly! When we refer to a language model as "large," we are essentially highlighting its scale and capabilities. Here are some key reasons why LLMs are called large:

1. Size of the Model: The most obvious reason is the sheer size of the model itself. LLMs typically involve complex neural network architectures with numerous layers, nodes, or parameters. These models require significant computational resources to train and run, making them "large" in terms of their complexity and scope.
2. Scale of Training Data: Another important aspect is the scale of the training data used to train LLMs. Modern language models are typically trained on vast amounts of text data, which can range from tens of thousands to millions or even billions of words. The larger the dataset, the more accurate and robust the model becomes.
3. Generative Capabilities: Large language models are designed to generate coherent and natural-sounding text. They have learned patterns and structures of language from 